In this note, we demonstrate how to build the full variant calling pipeline with GATK ```MUTECT2``` tool. For a detailed reference, refer to this article [(How to) Call somatic mutations using GATK4 Mutect2 (Deprecated)](https://gatk.broadinstitute.org/hc/en-us/articles/360035889791?id=11136#5). Of note, this tutorial is deprecated and can only be applied for GATK version 4.1.0 or earlier. For the updated version, refer to this note [Change-log in MUTECT2 variant calling pipeline](https://gatk.broadinstitute.org/hc/en-us/articles/360035531132). 

The version used in this note is GATK 4.0.12. 

Why not GATK 4.2? --> There are still some bugs with the creation of panel normals from database generated by the built-in import function of GATK. 

# Introduction to the MUTECT2 pipeline 

# Data summary

(last modified: 11/08/2021)

Up to now, we have applied this pipeline to the following dataset: 

- Liver data (LABCODE: LBH, LBHC, FLBH, BLBH): 50 pair samples, 96 Control samples  
- Colorectal data (LABCODE: CRCB, CRCC, CRCT, BCRCB): 41 pair samples, 73 Control samples

Generate a metadata ```.csv``` file for all samples. Since the LABCODE for each type of samples are different. We have to generate the metadata files separately.

For Colorectal data

```sh
path_to_datadir=/media/tronghieu/HNSD01/WORKING_DATA/CRC_BAM;
path_to_sample_index=/media/tronghieu/HNSD01/src_github/ECD-FRAGMENTATION-PROFILING/data/CRC_pairsample_idx.txt;
indices=$(cat $path_to_sample_index);
metadata=/media/tronghieu/HNSD01/src_github/ECD-FRAGMENTATION-PROFILING/data/CRC_sample_metadata.csv; 
touch $metadata;
echo -e "ID,LB,FFPE,WBC" >> $metadata;
for index in $indices;do \
            LB_file=$(ls $path_to_datadir/LB/*-CRCB${index}_*.bam);
            FFPE_file=$(ls $path_to_datadir/FFPE/*-CRCT${index}_*.bam);
            WBC_file=$(ls $path_to_datadir/WBC/*-BCRCB${index}_*.bam);
            echo -e Pair_$index,$LB_file,$FFPE_file,$WBC_file >> $metadata;
            done
```

For Liver data

```sh
path_to_datadir=/media/tronghieu/HNSD01/WORKING_DATA/LBH_BAM;
path_to_sample_index=/media/tronghieu/HNSD01/src_github/ECD-FRAGMENTATION-PROFILING/data/LBH_pairsample_idx.txt;
indices=$(cat $path_to_sample_index);
metadata=/media/tronghieu/HNSD01/src_github/ECD-FRAGMENTATION-PROFILING/data/LBH_sample_metadata.csv; 
touch $metadata;
echo -e "ID,LB,FFPE,WBC" >> $metadata;
for index in $indices;do \
            LB_file=$(ls $path_to_datadir/LB/*LBH${index}*.bam);
            FFPE_file=$(ls $path_to_datadir/FFPE/*FLBH${index}*.bam);
            WBC_file=$(ls $path_to_datadir/WBC/*BLBH${index}*.bam);
            echo -e Pair_$index,$LB_file,$FFPE_file,$WBC_file >> $metadata;
            done
```

# Main pipeline scripts